# Chatbot Agent Testing Notebook

This notebook provides a user-friendly interface for testing and experimenting with the Chatbot Agent.
It's designed for users with little programming experience.

## Features:
- Load and edit agent configuration settings
- Generate or load test questions
- Process questions through the Chatbot Agent
- Review and analyze results
- Export results with timestamps

## Getting Started:
1. Run cells in order from top to bottom
2. Edit configuration values as needed
3. Generate or load test questions
4. Review questions before processing
5. Run the agent and review results

In [ ]:
# Import required libraries
import json
import yaml
import os
import sys
from datetime import datetime
import pandas as pd
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Set the working directory to the root of the project
os.chdir('/workspace')

# Add workspace to path for imports (this helps with relative imports)
sys.path.insert(0, '/workspace')

# Import our system components
from src.nodes.chatbot_agent import ChatbotAgentNode
from src.core.config.agent_config_manager import AgentConfigManager
from src.integrations.llm_providers import LLMProviderFactory

print("✅ All libraries imported successfully!")
print("Ready to start testing the Chatbot Agent.")

## Step 1: Load Configuration Settings

The following cell loads the current configuration for the Chatbot Agent.
You can edit these values to customize the agent's behavior.

In [ ]:
# Load configuration from files
config_base_path = Path('/workspace/config')
agent_config_path = config_base_path / 'agents' / 'chatbot_agent'

def load_config_files():
    """Load all configuration files for the Chatbot Agent"""
    configs = {}
    
    # Load agent config
    with open(agent_config_path / 'config.yaml', 'r') as f:
        configs['agent'] = yaml.safe_load(f)
    
    # Load prompts
    with open(agent_config_path / 'prompts.yaml', 'r') as f:
        configs['prompts'] = yaml.safe_load(f)
    
    # Load models
    with open(agent_config_path / 'models.yaml', 'r') as f:
        configs['models'] = yaml.safe_load(f)
    
    # Load shared models for reference
    with open(config_base_path / 'shared' / 'models.yaml', 'r') as f:
        configs['shared_models'] = yaml.safe_load(f)
    
    return configs

# Load configurations
configs = load_config_files()

print("📁 Configuration files loaded successfully!")
print(f"Agent name: {configs['agent']['agent']['name']}")
print(f"Agent version: {configs['agent']['agent']['version']}")
print(f"Preferred model: {configs['agent']['models']['preferred']}")

## Step 2: Editable Configuration Settings

Edit these settings to customize how the Chatbot Agent behaves.
These variables map directly to the configuration files and can be exported later.

In [3]:
# EDITABLE CONFIGURATION VARIABLES
# These can be modified to customize the agent behavior

# === MODEL SETTINGS ===
agent_preferred_model = configs['agent']['models']['preferred']  # Which model to use first
agent_fallback_models = configs['agent']['models']['fallback']  # Backup models if primary fails
agent_temperature = configs['agent']['settings']['temperature']  # Creativity level (0.0-1.0)
agent_max_tokens = configs['agent']['settings']['max_tokens']  # Maximum response length
agent_timeout = configs['agent']['settings']['timeout']  # Timeout in seconds

# === BEHAVIOR SETTINGS ===
agent_response_style = configs['agent']['behavior']['response_style']  # How the agent responds
agent_context_integration = configs['agent']['behavior']['context_integration']  # Use conversation history
agent_personalization = configs['agent']['behavior']['personalization']  # Personalize responses

# === ESCALATION SETTINGS ===
agent_confidence_threshold = configs['agent']['escalation']['confidence_threshold']  # When to escalate (0.0-1.0)
agent_auto_escalation = configs['agent']['escalation']['enable_auto_escalation']  # Enable automatic escalation
agent_escalation_triggers = configs['agent']['escalation']['escalation_triggers']  # What triggers escalation

# === PROMPT STYLE ===
agent_prompt_style = configs['prompts']['response_guidelines']['style']  # Communication style
agent_include_empathy = configs['prompts']['response_guidelines']['include_empathy']  # Show empathy
agent_customer_focus = configs['prompts']['response_guidelines']['customer_focus']  # Focus on customer needs

# Display current literal values for easy editing
print("⚙️ Configuration variables loaded and ready for editing!")
print("\nCurrent literal values (you can copy these to edit the variables above):")
print(f"agent_preferred_model = '{agent_preferred_model}'")
print(f"agent_fallback_models = {agent_fallback_models}")
print(f"agent_temperature = {agent_temperature}")
print(f"agent_max_tokens = {agent_max_tokens}")
print(f"agent_timeout = {agent_timeout}")
print(f"agent_response_style = '{agent_response_style}'")
print(f"agent_context_integration = {agent_context_integration}")
print(f"agent_personalization = {agent_personalization}")
print(f"agent_confidence_threshold = {agent_confidence_threshold}")
print(f"agent_auto_escalation = {agent_auto_escalation}")
print(f"agent_escalation_triggers = {agent_escalation_triggers}")
print(f"agent_prompt_style = '{agent_prompt_style}'")
print(f"agent_include_empathy = {agent_include_empathy}")
print(f"agent_customer_focus = {agent_customer_focus}")

print("\n💡 Copy any of these lines above to edit the variables, then re-run this cell to apply changes.")
print("📝 After editing, the new values will be used when processing questions.")

⚙️ Configuration variables loaded and ready for editing!

Current literal values (you can copy these to edit the variables above):
agent_preferred_model = 'anthropic_general_budget'
agent_fallback_models = ['local_general_budget', 'local_general_standard']
agent_temperature = 0.7
agent_max_tokens = 2000
agent_timeout = 30
agent_response_style = 'clear_and_professional'
agent_context_integration = True
agent_personalization = True
agent_confidence_threshold = 0.7
agent_auto_escalation = True
agent_escalation_triggers = ['low_confidence', 'user_dissatisfaction', 'repeat_query']
agent_prompt_style = 'Warm, professional customer service tone'
agent_include_empathy = True
agent_customer_focus = True

💡 Copy any of these lines above to edit the variables, then re-run this cell to apply changes.
📝 After editing, the new values will be used when processing questions.


## Step 3: Input Generation or Loading

Choose whether to generate new test questions or load existing ones from a file.

In [4]:
# Create input generation interface
input_choice = widgets.RadioButtons(
    options=[('Generate new questions', 'generate'), ('Load from file', 'load')],
    description='Input source:',
    disabled=False
)

# Text area for describing what kind of inputs to generate
generation_prompt = widgets.Textarea(
    value="20 chatbot questions from users contacting their insurance company about their coverage. Some users are belligerent, some are confused and need simple explanations, some need information immediately about claims, some are asking about policy changes. Mix of simple and complex questions.",
    placeholder="Describe what kind of test questions you want...",
    description="Generation prompt:",
    layout=widgets.Layout(width='100%', height='100px')
)

# Model choice for generation
available_models = list(configs['shared_models']['model_aliases'].keys())
generation_model = widgets.Dropdown(
    options=available_models,
    value='anthropic_general_standard',
    description='Generation model:',
)

# File upload widget
file_upload = widgets.FileUpload(
    accept='.json,.txt,.csv',
    multiple=False,
    description='Upload file:'
)

print("📝 Input Configuration")
display(input_choice)
print("\nFor generation, describe your test scenario:")
display(generation_prompt)
display(generation_model)
print("\nFor file loading:")
display(file_upload)

📝 Input Configuration


RadioButtons(description='Input source:', options=(('Generate new questions', 'generate'), ('Load from file', …


For generation, describe your test scenario:


Textarea(value='20 chatbot questions from users contacting their insurance company about their coverage. Some …

Dropdown(description='Generation model:', index=1, options=('anthropic_general_budget', 'anthropic_general_sta…


For file loading:


FileUpload(value=(), accept='.json,.txt,.csv', description='Upload file:')

In [5]:
# Generate or load test questions
test_questions = []

def generate_test_questions(prompt, model_name, count=20):
    """Generate test questions using the specified model"""
    print(f"🤖 Generating test questions using {model_name}...")
    
    # Create a simple generation prompt
    generation_system_prompt = f"""
    Generate exactly {count} realistic customer service questions based on this scenario:
    {prompt}
    
    Return the questions as a JSON array with this format:
    [
        {{"id": 1, "question": "Question text here", "customer_type": "frustrated/confused/urgent/normal", "complexity": "simple/medium/complex"}},
        {{"id": 2, "question": "Another question", "customer_type": "normal", "complexity": "medium"}}
    ]
    
    Make the questions diverse and realistic. Include the customer emotional state and complexity level.
    IMPORTANT: Return ONLY the JSON array, no other text.
    """
    
    try:
        # Initialize model provider factory
        factory = LLMProviderFactory()
        provider = factory.create_provider(model_name)
        
        # Generate questions
        response = provider.generate_response(
            prompt="Generate the test questions now.",
            system_prompt=generation_system_prompt
        )
        
        print(f"📝 Raw response: {response[:200]}...")
        
        # Clean up the response - remove any markdown formatting
        cleaned_response = response.strip()
        if cleaned_response.startswith('```json'):
            cleaned_response = cleaned_response[7:]  # Remove ```json
        if cleaned_response.endswith('```'):
            cleaned_response = cleaned_response[:-3]  # Remove ```
        cleaned_response = cleaned_response.strip()
        
        # Parse JSON response
        questions_data = json.loads(cleaned_response)
        
        # Ensure we have a list
        if isinstance(questions_data, dict):
            questions_data = [questions_data]
        elif not isinstance(questions_data, list):
            raise ValueError(f"Expected list or dict, got {type(questions_data)}")
        
        # Validate each question has required fields
        validated_questions = []
        for i, q in enumerate(questions_data):
            if isinstance(q, dict):
                # Ensure required fields exist
                validated_q = {
                    "id": q.get("id", i + 1),
                    "question": q.get("question", f"Question {i + 1}"),
                    "customer_type": q.get("customer_type", "normal"),
                    "complexity": q.get("complexity", "medium")
                }
                validated_questions.append(validated_q)
            else:
                # Convert string to dict if needed
                validated_q = {
                    "id": i + 1,
                    "question": str(q),
                    "customer_type": "normal",
                    "complexity": "medium"
                }
                validated_questions.append(validated_q)
        
        return validated_questions
        
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        print(f"📝 Raw response that failed to parse: {response}")
        # Return sample questions as fallback
        return [
            {"id": 1, "question": "I need to file a claim for my car accident yesterday", "customer_type": "urgent", "complexity": "medium"},
            {"id": 2, "question": "What does my insurance cover exactly?", "customer_type": "confused", "complexity": "simple"},
            {"id": 3, "question": "Why did my premium go up this month? I haven't had any accidents!", "customer_type": "frustrated", "complexity": "medium"}
        ]
    except Exception as e:
        print(f"❌ Error generating questions: {e}")
        # Return sample questions as fallback
        return [
            {"id": 1, "question": "I need to file a claim for my car accident yesterday", "customer_type": "urgent", "complexity": "medium"},
            {"id": 2, "question": "What does my insurance cover exactly?", "customer_type": "confused", "complexity": "simple"},
            {"id": 3, "question": "Why did my premium go up this month? I haven't had any accidents!", "customer_type": "frustrated", "complexity": "medium"}
        ]

def load_questions_from_file(file_content, filename):
    """Load questions from uploaded file"""
    try:
        if filename.endswith('.json'):
            data = json.loads(file_content.decode('utf-8'))
            if isinstance(data, list):
                return data
            else:
                return [data]
        elif filename.endswith('.txt'):
            lines = file_content.decode('utf-8').strip().split('\n')
            return [{"id": i+1, "question": line.strip(), "customer_type": "normal", "complexity": "medium"} 
                   for i, line in enumerate(lines) if line.strip()]
        else:
            return [{"id": 1, "question": "Sample question from uploaded file", "customer_type": "normal", "complexity": "medium"}]
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return []

# Process based on user choice
if input_choice.value == 'generate':
    test_questions = generate_test_questions(
        generation_prompt.value, 
        generation_model.value
    )
    print(f"✅ Generated {len(test_questions)} test questions")
elif input_choice.value == 'load' and file_upload.value:
    uploaded_file = list(file_upload.value.values())[0]
    test_questions = load_questions_from_file(
        uploaded_file['content'], 
        uploaded_file['metadata']['name']
    )
    print(f"✅ Loaded {len(test_questions)} questions from file")
else:
    print("⚠️ Please select an input method and configure it above, then re-run this cell.")

# Display first few questions as preview
if test_questions:
    print("\n📋 Preview of test questions:")
    
    # Ensure test_questions is a list
    if isinstance(test_questions, list):
        for i, q in enumerate(test_questions[:3]):
            if isinstance(q, dict):
                print(f"  {i+1}. {q.get('question', 'No question text')} [{q.get('customer_type', 'unknown')}]")
            else:
                print(f"  {i+1}. {q} [type: {type(q)}]")
        if len(test_questions) > 3:
            print(f"  ... and {len(test_questions) - 3} more questions")
    else:
        print(f"  ⚠️ Expected list but got {type(test_questions)}: {test_questions}")
        # Convert to list if possible
        if hasattr(test_questions, '__iter__') and not isinstance(test_questions, (str, bytes)):
            test_questions = list(test_questions)
            print(f"  ✅ Converted to list with {len(test_questions)} items")
        else:
            test_questions = []
            print(f"  ❌ Could not convert to list, using empty list")

🤖 Generating test questions using anthropic_general_standard...
✅ Creating LLM provider: anthropic_general_standard → claude-3-5-sonnet-20241022 (anthropic)
✅ 12:58:29.964 [INFO    ] claude-3-5-sonnet-20241022 | Initializing LLM provider | model_name=claude-3-5-sonnet-20241022
✅ 12:58:29.967 [INFO    ] claude-3-5-sonnet-20241022 | LLM provider initialized successfully | model_name=claude-3-5-sonnet-20241022
✅ 12:58:54.050 [INFO    ] claude-3-5-sonnet-20241022 | Model call: claude-3-5-sonnet-20241022 - generate_response | model_name=claude-3-5-sonnet-20241022 operation=generate_response
📝 Raw response: [
    {"id": 1, "question": "I just had a car accident 10 minutes ago, what do I need to do right now??", "customer_type": "urgent", "complexity": "simple"},
    {"id": 2, "question": "Why the hell di...
✅ Generated 20 test questions

📋 Preview of test questions:
  1. I just had a car accident 10 minutes ago, what do I need to do right now?? [urgent]
  2. Why the hell did my premium go up

## Step 4: Review and Edit Questions

Review the loaded or generated questions and make any edits before processing.

In [6]:
# Create interactive question editor
if test_questions:
    print("📝 Question Editor - You can modify questions before processing")
    print("Edit the questions in the table below, then run the next cell to process them.\n")
    
    # Convert to DataFrame for easy editing
    df = pd.DataFrame(test_questions)
    
    # Ensure required columns exist
    if 'id' not in df.columns:
        df['id'] = range(1, len(df) + 1)
    if 'customer_type' not in df.columns:
        df['customer_type'] = 'normal'
    if 'complexity' not in df.columns:
        df['complexity'] = 'medium'
    
    # Display editable table
    print("Current questions (you can edit the JSON below if needed):")
    display(df)
    
    # Show JSON for manual editing if needed
    questions_json = widgets.Textarea(
        value=json.dumps(test_questions, indent=2),
        description="Questions JSON:",
        layout=widgets.Layout(width='100%', height='200px')
    )
    
    print("\nAdvanced: Edit questions as JSON (optional):")
    display(questions_json)
    
    def update_questions_from_json():
        """Update questions from the JSON editor"""
        global test_questions
        try:
            test_questions = json.loads(questions_json.value)
            print("✅ Questions updated from JSON editor")
        except Exception as e:
            print(f"❌ Error parsing JSON: {e}")
    
    # Button to update from JSON
    update_btn = widgets.Button(description="Update from JSON")
    update_btn.on_click(lambda b: update_questions_from_json())
    display(update_btn)
    
else:
    print("⚠️ No questions loaded. Please run the previous cell to generate or load questions.")

📝 Question Editor - You can modify questions before processing
Edit the questions in the table below, then run the next cell to process them.

Current questions (you can edit the JSON below if needed):


,id,question,customer_type,complexity
0,1,"I just had a car accident 10 minutes ago, what...",urgent,simple
1,2,Why the hell did my premium go up by $200? Thi...,frustrated,medium
2,3,I don't understand my deductible. Can you expl...,confused,simple
3,4,We're adding a teenage driver to our policy. W...,normal,simple
4,5,My house flooded but I can't find my policy nu...,urgent,medium
5,6,I've been a customer for 15 years and you're r...,frustrated,medium
6,7,Can you help me understand what 'comprehensive...,confused,medium
7,8,I need to file a claim for a multi-car acciden...,urgent,complex
8,9,How do I add my new boat to my existing policy?,normal,simple
9,10,I've been trying to get my claim processed for...,frustrated,complex



Advanced: Edit questions as JSON (optional):


Textarea(value='[\n  {\n    "id": 1,\n    "question": "I just had a car accident 10 minutes ago, what do I nee…

Button(description='Update from JSON', style=ButtonStyle())

## Step 5: Process Questions Through Chatbot Agent

Run the questions through the Chatbot Agent and collect results.

In [ ]:
# Process questions through the Chatbot Agent
if not test_questions:
    print("⚠️ No questions to process. Please load or generate questions first.")
else:
    print(f"🤖 Processing {len(test_questions)} questions through Chatbot Agent...")
    print(f"Using model: {agent_preferred_model}")
    print(f"Temperature: {agent_temperature}")
    print("\n" + "="*50)
    
    # Initialize the Chatbot Agent
    try:
        # Create config manager with the correct config directory
        config_manager = AgentConfigManager(config_dir='/workspace/config')
        
        # Create a simple context provider (using None for now)
        context_provider = None
        
        # Initialize Chatbot Agent
        chatbot_agent = ChatbotAgentNode(config_manager, context_provider)
        
        print("✅ Chatbot Agent initialized successfully")
        
    except Exception as e:
        print(f"❌ Error initializing Chatbot Agent: {e}")
        print("Continuing with mock responses for demonstration...")
        chatbot_agent = None
    
    # Process each question
    results = []
    
    for i, question_data in enumerate(test_questions):
        question_id = question_data.get('id', i + 1)
        question_text = question_data.get('question', '')
        customer_type = question_data.get('customer_type', 'normal')
        complexity = question_data.get('complexity', 'medium')
        
        print(f"\n🔄 Processing question {question_id}: {question_text[:60]}...")
        
        try:
            if chatbot_agent:
                # Create state for the Chatbot Agent
                from datetime import datetime
                state = {
                    'query': question_text,
                    'user_id': 'test_user',
                    'session_id': f'test_session_{i}',
                    'query_id': f'query_{question_id}',
                    'timestamp': datetime.now().isoformat(),
                    'messages': []
                }
                
                # Process through agent using __call__ method
                response_state = chatbot_agent(state)
                
                answer = response_state.get('ai_response', 'No response generated')
                confidence = response_state.get('initial_assessment', {}).get('confidence', 0.8)
                needs_escalation = confidence < agent_confidence_threshold
                needs_more_input = 'clarification' in answer.lower() or 'more information' in answer.lower()
                
            else:
                # Mock processing for demonstration
                import random
                answer = f"Thank you for your question about {question_text[:30]}... I'd be happy to help you with that. [This is a mock response for demonstration]"
                confidence = random.uniform(0.6, 0.95)
                needs_escalation = confidence < agent_confidence_threshold
                needs_more_input = random.random() < 0.1  # 10% chance
            
            # Create result entry
            result = {
                'id': question_id,
                'original_question': question_text,
                'customer_type': customer_type,
                'complexity': complexity,
                'ai_answer': answer,
                'confidence_score': confidence,
                'needs_escalation': needs_escalation,
                'needs_more_input': needs_more_input,
                'processing_time': datetime.now().isoformat(),
                'model_used': agent_preferred_model,
                'temperature': agent_temperature
            }
            
            results.append(result)
            
            # Show progress
            status = "🔄 Needs more input" if needs_more_input else ("⚠️ Escalation needed" if needs_escalation else "✅ Complete")
            print(f"   {status} (confidence: {confidence:.2f})")
            
        except Exception as e:
            print(f"   ❌ Error processing question: {e}")
            # Add error result
            result = {
                'id': question_id,
                'original_question': question_text,
                'customer_type': customer_type,
                'complexity': complexity,
                'ai_answer': f"Error processing question: {e}",
                'confidence_score': 0.0,
                'needs_escalation': True,
                'needs_more_input': False,
                'processing_time': datetime.now().isoformat(),
                'model_used': agent_preferred_model,
                'temperature': agent_temperature,
                'error': str(e)
            }
            results.append(result)
    
    print("\n" + "="*50)
    print(f"✅ Processing complete! Processed {len(results)} questions.")
    
    # Summary statistics
    total_questions = len(results)
    needs_more_input_count = sum(1 for r in results if r['needs_more_input'])
    needs_escalation_count = sum(1 for r in results if r['needs_escalation'])
    avg_confidence = sum(r['confidence_score'] for r in results) / total_questions if total_questions > 0 else 0
    
    print(f"\n📊 Summary:")
    print(f"  Total questions: {total_questions}")
    print(f"  Need more input: {needs_more_input_count} ({needs_more_input_count/total_questions*100:.1f}%)")
    print(f"  Need escalation: {needs_escalation_count} ({needs_escalation_count/total_questions*100:.1f}%)")
    print(f"  Average confidence: {avg_confidence:.2f}")

## Step 6: Handle Questions Needing More Input

Re-process questions that were flagged as needing more input from the user.

In [ ]:
# Handle questions that need more input
if 'results' in locals() and results:
    # Find questions that need more input
    questions_needing_input = [r for r in results if r['needs_more_input']]
    
    if questions_needing_input:
        print(f"🔄 Found {len(questions_needing_input)} questions needing more input.")
        print("Re-processing with additional context...\n")
        
        # Process questions needing more input
        for result in questions_needing_input:
            question_id = result['id']
            original_question = result['original_question']
            first_response = result['ai_answer']
            
            print(f"🔄 Re-processing question {question_id}...")
            
            try:
                if chatbot_agent:
                    # Create follow-up state
                    from datetime import datetime
                    follow_up_state = {
                        'query': 'Can you provide more specific information?',
                        'user_id': 'test_user',
                        'session_id': f'test_session_{question_id}',
                        'query_id': f'query_{question_id}_followup',
                        'timestamp': datetime.now().isoformat(),
                        'messages': [
                            {'role': 'user', 'content': original_question},
                            {'role': 'assistant', 'content': first_response}
                        ]
                    }
                    
                    # Re-process through agent using __call__ method
                    response_state = chatbot_agent(follow_up_state)
                    
                    follow_up_answer = response_state.get('ai_response', 'No follow-up response generated')
                    follow_up_confidence = response_state.get('initial_assessment', {}).get('confidence', 0.8)
                    
                else:
                    # Mock follow-up response
                    follow_up_answer = f"Thank you for asking for more details. Let me provide additional specific information about {original_question[:30]}... [Mock follow-up response]"
                    follow_up_confidence = 0.85
                
                # Add follow-up interaction to results
                follow_up_result = {
                    'id': f"{question_id}_followup",
                    'parent_id': question_id,
                    'original_question': 'Can you provide more specific information?',
                    'customer_type': result['customer_type'],
                    'complexity': result['complexity'],
                    'ai_answer': follow_up_answer,
                    'confidence_score': follow_up_confidence,
                    'needs_escalation': follow_up_confidence < agent_confidence_threshold,
                    'needs_more_input': False,  # Assume follow-up resolves the need
                    'processing_time': datetime.now().isoformat(),
                    'model_used': agent_preferred_model,
                    'temperature': agent_temperature,
                    'is_followup': True
                }
                
                results.append(follow_up_result)
                
                # Update original result to show it no longer needs more input
                result['needs_more_input'] = False
                result['has_followup'] = True
                
                print(f"   ✅ Follow-up processed (confidence: {follow_up_confidence:.2f})")
                
            except Exception as e:
                print(f"   ❌ Error processing follow-up: {e}")
        
        print(f"\n✅ Follow-up processing complete. Added {len(questions_needing_input)} follow-up interactions.")
        
    else:
        print("✅ No questions needed additional input. All questions processed successfully.")
        
else:
    print("⚠️ No results to process. Please run the previous cells first.")

## Step 7: Export Results and Settings

Save the results and configuration settings to files with timestamps.

In [ ]:
# Export results and settings
if 'results' in locals() and results:
    # Create timestamp for filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    
    # Create experiment_runs directory if it doesn't exist
    experiment_dir = Path('/workspace/notebooks/experiment_runs')
    experiment_dir.mkdir(exist_ok=True)
    
    # Export results
    results_filename = f"chatbot_agent_output_{timestamp}.json"
    results_path = experiment_dir / results_filename
    
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"📄 Results exported to: {results_path}")
    
    # Export settings
    settings_filename = f"chatbot_agent_settings_{timestamp}.json"
    settings_path = experiment_dir / settings_filename
    
    settings_export = {
        'experiment_info': {
            'timestamp': timestamp,
            'agent_type': 'chatbot_agent',
            'total_questions': len([r for r in results if not r.get('is_followup', False)]),
            'total_interactions': len(results),
            'follow_up_count': len([r for r in results if r.get('is_followup', False)])
        },
        'model_settings': {
            'preferred_model': agent_preferred_model,
            'fallback_models': agent_fallback_models,
            'temperature': agent_temperature,
            'max_tokens': agent_max_tokens,
            'timeout': agent_timeout
        },
        'behavior_settings': {
            'response_style': agent_response_style,
            'context_integration': agent_context_integration,
            'personalization': agent_personalization,
            'prompt_style': agent_prompt_style,
            'include_empathy': agent_include_empathy,
            'customer_focus': agent_customer_focus
        },
        'escalation_settings': {
            'confidence_threshold': agent_confidence_threshold,
            'auto_escalation': agent_auto_escalation,
            'escalation_triggers': agent_escalation_triggers
        },
        'generation_settings': {
            'input_method': input_choice.value if 'input_choice' in locals() else 'unknown',
            'generation_prompt': generation_prompt.value if 'generation_prompt' in locals() else 'N/A',
            'generation_model': generation_model.value if 'generation_model' in locals() else 'N/A'
        }
    }
    
    with open(settings_path, 'w') as f:
        json.dump(settings_export, f, indent=2)
    
    print(f"⚙️ Settings exported to: {settings_path}")
    
    # Show summary
    print(f"\n📊 Export Summary:")
    print(f"  Timestamp: {timestamp}")
    print(f"  Results file: {results_filename}")
    print(f"  Settings file: {settings_filename}")
    print(f"  Total interactions: {len(results)}")
    print(f"  Original questions: {len([r for r in results if not r.get('is_followup', False)])}")
    print(f"  Follow-up interactions: {len([r for r in results if r.get('is_followup', False)])}")
    print(f"  Files saved to: {experiment_dir.absolute()}")
    
else:
    print("⚠️ No results to export. Please process questions first.")

## Step 8: Review Results

Display and analyze the final results.

In [10]:
# Review and analyze results
if 'results' in locals() and results:
    print("📋 Results Review and Analysis\n")
    
    # Separate original questions and follow-ups
    original_results = [r for r in results if not r.get('is_followup', False)]
    followup_results = [r for r in results if r.get('is_followup', False)]
    
    # Create DataFrame for analysis
    df_results = pd.DataFrame(original_results)
    
    print("=== SUMMARY STATISTICS ===")
    print(f"Total original questions: {len(original_results)}")
    print(f"Follow-up interactions: {len(followup_results)}")
    print(f"Average confidence score: {df_results['confidence_score'].mean():.3f}")
    print(f"Questions needing escalation: {df_results['needs_escalation'].sum()} ({df_results['needs_escalation'].sum()/len(df_results)*100:.1f}%)")
    print(f"Questions that needed more input: {df_results['needs_more_input'].sum()}")
    
    # Analysis by customer type
    print("\n=== ANALYSIS BY CUSTOMER TYPE ===")
    customer_analysis = df_results.groupby('customer_type').agg({
        'confidence_score': 'mean',
        'needs_escalation': 'sum',
        'needs_more_input': 'sum'
    }).round(3)
    print(customer_analysis)
    
    # Analysis by complexity
    print("\n=== ANALYSIS BY COMPLEXITY ===")
    complexity_analysis = df_results.groupby('complexity').agg({
        'confidence_score': 'mean',
        'needs_escalation': 'sum',
        'needs_more_input': 'sum'
    }).round(3)
    print(complexity_analysis)
    
    # Show detailed results table
    print("\n=== DETAILED RESULTS ===")
    
    # Create a simplified view for display
    display_columns = ['id', 'customer_type', 'complexity', 'confidence_score', 'needs_escalation', 'needs_more_input']
    display_df = df_results[display_columns].copy()
    display_df['confidence_score'] = display_df['confidence_score'].round(3)
    
    print("Summary table:")
    display(display_df)
    
    # Show a few sample Q&A pairs
    print("\n=== SAMPLE QUESTION & ANSWER PAIRS ===")
    for i, result in enumerate(original_results[:3]):
        print(f"\n--- Question {result['id']} ({result['customer_type']}, {result['complexity']}) ---")
        print(f"Q: {result['original_question']}")
        print(f"A: {result['ai_answer'][:200]}{'...' if len(result['ai_answer']) > 200 else ''}")
        print(f"Confidence: {result['confidence_score']:.3f} | Escalation: {result['needs_escalation']} | More input: {result['needs_more_input']}")
        
        # Show follow-up if it exists
        followup = next((f for f in followup_results if f.get('parent_id') == result['id']), None)
        if followup:
            print(f"\n  Follow-up Q: {followup['original_question']}")
            print(f"  Follow-up A: {followup['ai_answer'][:200]}{'...' if len(followup['ai_answer']) > 200 else ''}")
            print(f"  Follow-up Confidence: {followup['confidence_score']:.3f}")
    
    print("\n" + "="*60)
    print("✅ Review complete! Check the exported files for full details.")
    
else:
    print("⚠️ No results to review. Please process questions first.")

📋 Results Review and Analysis

=== SUMMARY STATISTICS ===
Total original questions: 20
Follow-up interactions: 0
Average confidence score: 0.000
Questions needing escalation: 20 (100.0%)
Questions that needed more input: 0

=== ANALYSIS BY CUSTOMER TYPE ===
               confidence_score  needs_escalation  needs_more_input
customer_type                                                      
confused                    0.0                 5                 0
frustrated                  0.0                 5                 0
normal                      0.0                 4                 0
urgent                      0.0                 6                 0

=== ANALYSIS BY COMPLEXITY ===
            confidence_score  needs_escalation  needs_more_input
complexity                                                      
complex                  0.0                 4                 0
medium                   0.0                 9                 0
simple                   0.0              

,id,customer_type,complexity,confidence_score,needs_escalation,needs_more_input
0,1,urgent,simple,0.0,True,False
1,2,frustrated,medium,0.0,True,False
2,3,confused,simple,0.0,True,False
3,4,normal,simple,0.0,True,False
4,5,urgent,medium,0.0,True,False
5,6,frustrated,medium,0.0,True,False
6,7,confused,medium,0.0,True,False
7,8,urgent,complex,0.0,True,False
8,9,normal,simple,0.0,True,False
9,10,frustrated,complex,0.0,True,False



=== SAMPLE QUESTION & ANSWER PAIRS ===

--- Question 1 (urgent, simple) ---
Q: I just had a car accident 10 minutes ago, what do I need to do right now??
A: Error processing question: 'NoneType' object has no attribute 'get_context_summary'
Confidence: 0.000 | Escalation: True | More input: False

--- Question 2 (frustrated, medium) ---
Q: Why the hell did my premium go up by $200? This is ridiculous and I want answers NOW!
A: Error processing question: 'NoneType' object has no attribute 'get_context_summary'
Confidence: 0.000 | Escalation: True | More input: False

--- Question 3 (confused, simple) ---
Q: I don't understand my deductible. Can you explain what that means in simple terms?
A: Error processing question: 'NoneType' object has no attribute 'get_context_summary'
Confidence: 0.000 | Escalation: True | More input: False

✅ Review complete! Check the exported files for full details.


## Next Steps

🎉 **Congratulations!** You've successfully tested the Chatbot Agent.

### What you've accomplished:
- ✅ Loaded and customized Chatbot Agent configuration
- ✅ Generated or loaded test questions
- ✅ Processed questions through the Chatbot Agent
- ✅ Handled follow-up interactions for unclear responses
- ✅ Exported results and settings with timestamps
- ✅ Analyzed performance by customer type and complexity

### Your exported files contain:
- **Results file**: All questions, answers, confidence scores, and flags
- **Settings file**: Complete configuration used for this experiment

### To import settings to config files:
The exported settings use variable names that map directly to the configuration structure.
You can use these files to:
1. Recreate successful experiments
2. Import optimized settings back to the main config files
3. Compare different configuration approaches

### Ready for more testing?
- Modify the configuration variables in Step 2 and re-run
- Try different models or temperature settings
- Test with different types of customer questions
- Experiment with different confidence thresholds

---
*This notebook is part of the Human-in-the-Loop AI System for customer service optimization.*